In [1]:
# To generate a PDF output (box scores look ugly):
# jupyter nbconvert --to pdf --template notebooks/hidecode notebooks/top5.ipynb

# To generate a HTML:
# jupyter nbconvert --no-input --to html notebooks/top5.ipynb

In [2]:
import datetime as dt
import functools
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import display, Markdown


In [3]:
# Configuration

raw_data_dir = Path("../../raw_data")
reports_dir = Path("../../reports")
season_id = 25
week_date = dt.datetime(2019, 10, 17)
season_raw_data_dir = raw_data_dir / f"season{season_id}"
season_reports_dir = reports_dir / f"season{season_id}"
season_box_scores_dir = raw_data_dir / f"season{season_id}" / "box_scores"

In [4]:
display(Markdown("# thegame.report"))
display(Markdown(f"Season {season_id}, week of {week_date.strftime('%Y-%m-%d')}"))
display(Markdown(f"Report generated at {dt.datetime.utcnow()}"))
display(Markdown("### Contents"))
display(Markdown(" 1. Labels"))
display(Markdown(" 2. This Week's Box Scores"))
display(Markdown(" 3. Stat Leaders (Teams and Individuals)"))

# thegame.report

Season 25, week of 2019-10-17

Report generated at 2019-10-18 21:02:12.895247

### Contents

 1. Labels

 2. This Week's Box Scores

 3. Stat Leaders (Teams and Individuals)

In [5]:
measure_descriptions = pd.read_csv(raw_data_dir / "stat_descriptions.csv")
display(Markdown("## 1. Labels"))

## 1. Labels

In [6]:
display(measure_descriptions[["abbreviation", "description"]])

,abbreviation,description
0,PTS,Points
1,FGM,Field goals made
2,FGA,Field goals attempted
3,FG%,Field goal percentage
4,3PM,Three point shots made
5,3PA,Three point shots attempted
6,3P%,Three point shot percentage
7,FTM,Free throws made
8,FTA,Free throws attempted
9,FT%,Free throw percentage


In [7]:

stats_groups = [
    [
        "Performance Index Rating",
        ["pir", "pts", "reb", "ast", "tov", "stl", "blk", "fc"],
        [False, False, False, False, True, False, False, True],
    ],
    [
        "Points",
        ["pts", "fg%", "fgm", "fga", "3pm", "3pa", "ftm", "fta"],
        [False, False, False, True, False, True, False, True],
    ],    
    [
        "Rebounds",
        ["reb", "oreb", "dreb"],
        [False, False, False],
    ],
    [
        "Offensive Rebounds",
        ["oreb", "dreb", "reb"],
        [False, False, False],
    ],
    [
        "Assists",
        ["ast", "ast_tov_ratio", "tov"],
        [False, False, True],
    ],
    [
        "Steals",
        ["stl", "tov", "fc"],
        [False, True, True],
    ],
    [
        "Blocks",
        ["blk",],
        [False,],
    ],
    [
        "Turnovers",
        ["tov",],
        [False,],
    ],
    [
        "Fouls Committed",
        ["fc", "fd",],
        [False, True],
    ],
    [
        "Field Goal Percentage",
        ["fg%", "fgm", "fga"],
        [False, False, True],
    ],
    [
        "Three Point Shot Percentage",
        ["3p%", "3pm", "3pa"],
        [False, False, True],
    ],
    [
        "Free Throw Percentage",
        ["ft%", "ftm", "fta"],
        [False, False, True],
    ]
]


def ast_tov_ratio(r):
    if not r["tov"]:
        return r["ast"]
    return round(10.0 * r["ast"] / r["tov"]) / 10


def percentage(r, prefix):
    if not r[f"{prefix}a"]:
        return 0
    return round(1000.0 * r[f"{prefix}m"] / r[f"{prefix}a"]) / 10



calculated_measures = {
    "fg%": functools.partial(percentage, prefix="fg"),
    "3p%": functools.partial(percentage, prefix="3p"),
    "ft%": functools.partial(percentage, prefix="ft"),
    "ast_tov_ratio": ast_tov_ratio,
}


enabled_measures = [
    "pts",
    "fgm",
    "fga",
    "fg%",
    "3pm",
    "3pa",
    "3p%",
    "ftm",
    "fta",
    "ft%",
    "oreb",
    "dreb",
    "reb",
    "ast",
    "stl",
    "tov",
    "blk",
    "blka",
    "fc",
    "fd",
    "pir",
    "ast_tov_ratio",
]

measure_labels = {
    "ast_tov_ratio": "AST / TOV",
    "player_gender": "g",
}


def fix_column_names(measure_names):
    fixed = []
    for c in measure_names:
        if c in measure_labels:
            fixed.append(measure_labels[c])
            continue
        if c.endswith("_name"):
            c = c[:-5]
        c = c.replace("_", " ")
        fixed.append(c.upper())
    return fixed



plain_measures = [s for s in enabled_measures if s not in calculated_measures]
calculated_measures_sorted = [s for s in enabled_measures if s in calculated_measures]



In [8]:
# Box scores of this week's games

display(Markdown("## 2. This Week's Box Scores"))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

for p in season_box_scores_dir.iterdir():
    g_date = dt.datetime.strptime(p.stem[:8], "%Y%m%d").date()
    if g_date != week_date.date():
        continue
    box_score = pd.read_csv(p)
    selected_columns = [c for c in box_score.columns if c not in ("player_id",)]
    box_score = box_score[selected_columns]
    box_score = box_score.replace(np.nan, "", regex=True)
    box_score.columns = fix_column_names(box_score.columns)
    display(box_score)

## 2. This Week's Box Scores

,TEAM,g,PLAYER,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,FC,FD,PIR
0,Green Machine,,,34,13,44,29.5,2,13,15.4,6,15,40.0,13,18,31,4,9,6,1,0,6,0,21
1,Green Machine,m,Ahmed,7,2,8,25.0,1,5,20.0,2,5,40.0,1,1,2,1,3,1,0,0,1,0,-2
2,Green Machine,m,AlexGM,7,3,6,50.0,0,0,0.0,1,4,25.0,4,10,14,2,0,3,1,0,4,0,17
3,Green Machine,f,Kelli,0,0,7,0.0,0,1,0.0,0,0,0.0,2,0,2,1,2,0,0,0,0,0,-6
4,Green Machine,f,Victoria,2,1,1,100.0,0,0,0.0,0,0,0.0,2,2,4,0,1,0,0,0,0,0,5
5,Green Machine,m,Basil,11,4,11,36.4,0,1,0.0,3,6,50.0,4,2,6,0,3,2,0,0,1,0,5
6,Green Machine,m,Jayd,7,3,10,30.0,1,5,20.0,0,0,0.0,0,1,1,0,0,0,0,0,0,0,1
7,Green Machine,f,Steph-Kansas,0,0,1,0.0,0,1,0.0,0,0,0.0,0,2,2,0,0,0,0,0,0,0,1
8,Lions,,,39,15,49,30.6,3,12,25.0,6,9,66.7,17,16,33,8,10,4,0,0,7,0,30
9,Lions,m,Diego,4,2,4,50.0,0,1,0.0,0,0,0.0,2,8,10,5,4,2,0,0,0,0,15


,TEAM,g,PLAYER,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,FC,FD,PIR
0,Camden Hells,,,34,15,54,27.8,1,18,5.6,3,4,75.0,5,20,25,8,6,5,4,0,3,0,27
1,Camden Hells,m,Sean,8,3,6,50.0,0,0,0.0,2,2,100.0,0,1,1,2,1,1,0,0,1,0,7
2,Camden Hells,f,Steph-Kansas,2,1,4,25.0,0,1,0.0,0,0,0.0,0,3,3,0,0,0,0,0,0,0,2
3,Camden Hells,m,Rob,15,7,20,35.0,0,6,0.0,1,2,50.0,1,4,5,0,2,1,1,0,1,0,5
4,Camden Hells,m,Ahmed,5,2,13,15.4,1,8,12.5,0,0,0.0,1,7,8,2,2,1,3,0,1,0,5
5,Camden Hells,f,Jess,0,0,4,0.0,0,2,0.0,0,0,0.0,1,1,2,2,1,1,0,0,0,0,0
6,Camden Hells,f,Kata,4,2,6,33.3,0,1,0.0,0,0,0.0,1,3,4,0,0,1,0,0,0,0,5
7,Camden Hells,m,Basil,0,0,1,0.0,0,0,0.0,0,0,0.0,1,1,2,2,0,0,0,0,0,0,3
8,Lost Angels,,,56,23,56,41.1,7,26,26.9,3,3,100.0,9,28,37,17,8,2,3,0,5,0,69
9,Lost Angels,m,Aldis,17,7,20,35.0,3,11,27.3,0,0,0.0,2,9,11,4,3,0,0,0,2,0,14


,TEAM,g,PLAYER,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,FC,FD,PIR
0,Hot Shots,,,22,6,31,19.4,2,6,33.3,8,15,53.3,5,16,21,5,17,3,1,0,11,16,8
1,Hot Shots,f,Marine,0,0,1,0.0,0,0,0.0,0,0,0.0,2,2,4,0,0,0,0,0,0,1,4
2,Hot Shots,f,Maliena,0,0,2,0.0,0,0,0.0,0,0,0.0,0,0,0,0,1,0,0,0,0,0,-3
3,Hot Shots,m,Keir,0,0,2,0.0,0,0,0.0,0,2,0.0,1,0,1,1,2,2,0,0,0,1,-1
4,Hot Shots,f,Weronika,0,0,1,0.0,0,0,0.0,0,0,0.0,0,2,2,3,1,0,0,0,0,3,6
5,Hot Shots,m,Marty,7,3,8,37.5,0,0,0.0,1,1,100.0,1,3,4,0,3,0,0,0,1,2,4
6,Hot Shots,m,David M,1,0,6,0.0,0,2,0.0,1,2,50.0,0,2,2,0,1,1,0,0,2,2,-4
7,Hot Shots,m,Harrison38,14,3,10,30.0,2,4,50.0,6,10,60.0,1,4,5,0,6,0,1,0,5,7,5
8,Hot Shots,m,Sharad,0,0,1,0.0,0,0,0.0,0,0,0.0,0,2,2,1,3,0,0,0,1,0,-2
9,Hot Shots,m,Daniel,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0


,TEAM,g,PLAYER,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,FC,FD,PIR
0,Burritos,,,50,22,44,50.0,1,5,20.0,5,10,50.0,9,27,36,14,7,5,3,0,4,0,70
1,Burritos,m,AlexConn,6,3,4,75.0,0,0,0.0,0,0,0.0,3,6,9,3,1,1,0,0,0,0,17
2,Burritos,f,Armelle,0,0,2,0.0,0,0,0.0,0,0,0.0,2,1,3,0,0,0,0,0,1,0,0
3,Burritos,f,Suzi,0,0,4,0.0,0,1,0.0,0,0,0.0,2,0,2,2,0,1,0,0,0,0,1
4,Burritos,f,Sylwia,7,3,9,33.3,0,0,0.0,1,2,50.0,0,1,1,0,0,0,0,0,0,0,1
5,Burritos,m,Wouter,23,10,13,76.9,0,0,0.0,3,7,42.9,2,9,11,2,2,0,3,0,3,0,27
6,Burritos,m,Tom,5,2,5,40.0,1,4,25.0,0,0,0.0,0,7,7,7,4,2,0,0,0,0,14
7,Burritos,f,Mary,9,4,7,57.1,0,0,0.0,1,1,100.0,0,3,3,0,0,1,0,0,0,0,10
8,North L Lions,,,29,12,47,25.5,3,12,25.0,2,4,50.0,7,11,18,9,8,7,0,0,8,0,10
9,North L Lions,u,Tanky,13,5,15,33.3,1,5,20.0,2,2,100.0,2,5,7,2,2,0,0,0,3,0,7


In [9]:
display(Markdown("## 3. Stats Leaders (Teams and Individuals)"))

player_logs = pd.read_csv(season_raw_data_dir / "player_logs.csv")
team_logs = pd.read_csv(season_raw_data_dir / "team_logs.csv")


player_totals = []
for (team_id, player_id, player_gender), group in player_logs.groupby(["team_id", "player_id", "player_gender"]):
    pt = {
        "team_id": team_id,
        "team_name": group["team_name"].iloc[0],
        "player_id": player_id,
        "player_gender": player_gender,
        "player_name": group["player_name"].iloc[0],
        **{s: group[s].sum() for s in plain_measures},
        **{f"{s}_avg": group[s].mean() for s in plain_measures},
    }
    
    for s in calculated_measures_sorted:
        pt[s] = calculated_measures[s](pt)
    
    player_totals.append(pt)

team_totals = []
for team_id, group in team_logs.groupby("team_id"):
    tt = {
        "team_id": team_id,
        "team_name": group["team_name"].iloc[0],
        **{s: group[s].sum() for s in plain_measures},
        **{f"{s}_avg": group[s].mean() for s in plain_measures},
    }
    
    for s in calculated_measures_sorted:
        tt[s] = calculated_measures[s](tt)
    
    team_totals.append(tt)
    
ptdf = pd.DataFrame(player_totals)
ttdf = pd.DataFrame(team_totals)

for title, sort_fields, sort_ascending in stats_groups:
    display(Markdown(f"### {title}"))
    
    teams_sorted_by_field = ttdf.sort_values(
        sort_fields,
        ascending=sort_ascending,
    )
    top8_by_field = teams_sorted_by_field[["team_name", *sort_fields]].head(8)
    top8_by_field.reset_index(drop=True, inplace=True)
    top8_by_field.index += 1
    top8_by_field.columns = fix_column_names(top8_by_field.columns)
    display(top8_by_field)

    for gender in ("f", "m"):
        this_gender = ptdf["player_gender"] == gender
        sorted_by_field = ptdf[this_gender].sort_values(
            sort_fields,
            ascending=sort_ascending,
        )
        top5_by_field = sorted_by_field[["player_name", "team_name", *sort_fields]].head(5)
        top5_by_field.reset_index(drop=True, inplace=True)
        top5_by_field.index += 1
        top5_by_field.columns = fix_column_names(top5_by_field.columns)
        display(top5_by_field)


## 3. Stats Leaders (Teams and Individuals)

### Performance Index Rating

,TEAM,PIR,PTS,REB,AST,TOV,STL,BLK,FC
1,Rockets,94,70,45,20,3,10,0,16
2,Burritos,70,50,36,14,7,5,3,4
3,Lost Angels,69,56,37,17,8,2,3,5
4,Lions,30,39,33,8,10,4,0,7
5,Camden Hells,27,34,25,8,6,5,4,3
6,Green Machine,21,34,31,4,9,6,1,6
7,North L Lions,10,29,18,9,8,7,0,8
8,Hot Shots,8,22,21,5,17,3,1,11


,PLAYER,TEAM,PIR,PTS,REB,AST,TOV,STL,BLK,FC
1,Janis,Rockets,13,9,3,7,0,2,0,3
2,Mary,Burritos,10,9,3,0,0,1,0,0
3,Sophie,Lost Angels,10,6,4,3,1,0,1,0
4,Daria,Lost Angels,8,9,3,1,0,1,0,0
5,Weronika,Hot Shots,6,0,2,3,1,0,0,0


,PLAYER,TEAM,PIR,PTS,REB,AST,TOV,STL,BLK,FC
1,Wouter,Burritos,27,23,11,2,2,0,3,3
2,Marc,Rockets,23,17,5,5,0,3,0,2
3,Rolands,Lost Angels,23,10,15,4,2,1,2,3
4,Darnell,Rockets,19,17,7,3,0,0,0,1
5,AlexGM,Green Machine,17,7,14,2,0,3,1,4


...

,PLAYER,TEAM,3P%,3PM,3PA
1,Harrison38,Hot Shots,50.0,2,4
2,Rolands,Lost Angels,50.0,1,2
3,Marc,Rockets,50.0,1,2
4,Rashid,Lions,28.6,2,7
5,Aldis,Lost Angels,27.3,3,11


### Free Throw Percentage

,TEAM,FT%,FTM,FTA
1,Lost Angels,100.0,3,3
2,Camden Hells,75.0,3,4
3,Lions,66.7,6,9
4,Rockets,61.5,8,13
5,Hot Shots,53.3,8,15
6,Burritos,50.0,5,10
7,North L Lions,50.0,2,4
8,Green Machine,40.0,6,15


,PLAYER,TEAM,FT%,FTM,FTA
1,Telle,Lions,100.0,2,2
2,Janis,Rockets,100.0,2,2
3,Mary,Burritos,100.0,1,1
4,Sylwia,Burritos,50.0,1,2
5,Kata,Lions,40.0,2,5


,PLAYER,TEAM,FT%,FTM,FTA
1,Rolands,Lost Angels,100.0,3,3
2,Sean,Camden Hells,100.0,2,2
3,Gus,Lions,100.0,2,2
4,Marty,Hot Shots,100.0,1,1
5,Harrison38,Hot Shots,60.0,6,10
